## ORM learn

In [ ]:
from sqlalchemy import create_engine
import MySQLdb

engine = create_engine("mysql://root:@localhost:3306/fish", echo=True)

In [ ]:
# connection engine
conn = engine.connect()
# conn.execute() and engine.execute() are same

In [ ]:
# sessions
from sqlalchemy.orm import sessionmaker
# session makes middle-ground between objects in Python and the engine
# that actually communicates with the database

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# creating tables
# Each class will be a table in our db and each attribute
# will be a column in this table. To map which table in 
# the db will be related to each class in our files, we 
# will use a SQLAlchemy system called Declarative


from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    password = Column(String(255))
    
    def __repr__(self):
        return f'User {self.name}'
    

# creating table in database
Base.meta.create_all(engine)

In [ ]:
# adding new records

user = User(name="Pujan Dahal", password="12345678")
session.add(user)

print(user.id) # None because the new record is only flushed i.e. recorded
# for commit but not actually committed

session.commit() # it commits all previous pending operations that were flushed
print(user.id) # 9

In [ ]:
# querying table

query = session.query(User).filter_by(name="Pujan Dahal")
query.count() # 2

query = session.query(User).filter(User.name=="John Snow").all()


In [ ]:
# creating new table after initial create_all

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    

Product.__table__.create(engine) # creates new table in database

In [ ]:
# creating foreign key relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship

Base = declarative_base()

class Product(Base):
    __tablename__ = 'product'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    user_id = Column(Integer, ForeignKey('user.id'))
    user = relationship('User')
    
    def __repr__(self):
        return f"Product {self.name}"
        
    
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    password = Column(String(255))
    products = relationship(Product, backref='users')
    
    
    def __repr__(self):
        return f"User {user.name}"
        

# create tables
Base.metadata.create_all(engine)

user = User(name='John')
product = Product(name='fogg', user=user)

session.add_all([user, product])
session.commit()

## ORM Practice

### Create engine

In [1]:
# importing create_engine from sqlalchemy to interact with our database
from sqlalchemy import create_engine, inspect

# create an engine that communicates with your database
engine = create_engine("mysql://root:@127.0.0.1:3306/ORG")
conn = engine.connect() # a connection to the engine which we will later use to execute commands on the database

# engine.table_names() is deprecated, so we have to use an additional entity called inspector
inspector = inspect(engine)
print(inspector.get_table_names()) # ['Bonus', 'Title', 'WORKER', 'Worker', 'WorkerClone']: this is the current list of tables

['Bonus', 'Title', 'Worker', 'WorkerClone']


### Dropping a table

In [ ]:
# dropping tables WORKER and WorkerClone
# we need to create metadata object that associates with database
# The MetaData object contains all of the schema constructs we’ve associated with it.

from sqlalchemy import MetaData, Table, select

metadata = MetaData()
WORKER = Table('WORKER', metadata, autoload=True, autoload_with=engine)

stmt = select([WORKER])
results_proxy = conn.execute(stmt) # this gives results_proxy
results_proxy.rowcount # 5 because there are 5 records in the table
for records in results_proxy.fetchall():
    print(records)
# (2, 'Niharika', 'Verma', 80000, datetime.datetime(2014, 6, 11, 9, 0), 'Admin')
# (3, 'Vishal', 'Singhal', 300000, datetime.datetime(2014, 2, 20, 9, 0), 'HR')
# (4, 'Amitabh', 'Singh', 500000, datetime.datetime(2014, 2, 20, 9, 0), 'Admin')
# (7, 'Satish', 'Kumar', 75000, datetime.datetime(2014, 1, 20, 9, 0), 'Account')
# (8, 'Geetika', 'Chauhan', 90000, datetime.datetime(2014, 4, 11, 9, 0), 'Admin')

WORKER.drop(engine)

In [6]:
inspector.get_table_names() # ['Bonus', 'Title', 'Worker', 'WorkerClone']

['Bonus', 'Title', 'Worker', 'WorkerClone']

In [ ]:
WorkerClone = Table('WorkerClone', metadata, autoload=True, autoload_with=engine)
WorkerClone.drop(engine)

In [15]:
inspector = inspect(engine)
inspector.get_table_names() # ['Bonus', 'Title', 'Worker']

['Bonus', 'Title', 'Worker']

### Querying tables

In [19]:
# Display id, firstname and salary of all records in Worker

worker = Table('Worker', metadata, autoload=True, autoload_with=engine)

stmt = select([worker])
# to get the list of names of columns
first_record = conn.execute(stmt).first()

first_record.keys()
# ['WORKER_ID',
#  'FIRST_NAME',
#  'LAST_NAME',
#  'SALARY',
#  'JOINING_DATE',
#  'DEPARTMENT']

for result in conn.execute(stmt).fetchall():
    print(result.WORKER_ID, result.FIRST_NAME, result.SALARY)

# 1 Monika 100000
# 2 Niharika 80000
# 3 Vishal 300000
# 4 Amitabh 500000
# 5 Vivek 500000
# 6 Vipul 200000
# 7 Satish 75000
# 8 Geetika 90000

1 Monika 100000
2 Niharika 80000
3 Vishal 300000
4 Amitabh 500000
5 Vivek 500000
6 Vipul 200000
7 Satish 75000
8 Geetika 90000


In [22]:
# display worker with max salary
from sqlalchemy import desc

stmt = select([worker])
stmt = stmt.order_by(desc(worker.columns.SALARY))
stmt = stmt.limit(1)

print(conn.execute(stmt).fetchall())
# [(4, 'Amitabh', 'Singh', 500000, datetime.datetime(2014, 2, 20, 9, 0), 'Admin')]

[(4, 'Amitabh', 'Singh', 500000, datetime.datetime(2014, 2, 20, 9, 0), 'Admin')]


In [25]:
# count number of worker with salary >= 100000
from sqlalchemy import func

stmt = select([func.count(worker.columns.WORKER_ID)])
stmt = stmt.where(worker.columns.SALARY >= 100000)

print(conn.execute(stmt).scalar()) # 5

# print all records with salary >= 10000
stmt = select([worker])
stmt = stmt.where(worker.columns.SALARY >= 100000)

for record in conn.execute(stmt).fetchall():
    print(record)
# (1, 'Monika', 'Arora', 100000, datetime.datetime(2014, 2, 20, 9, 0), 'HR')
# (3, 'Vishal', 'Singhal', 300000, datetime.datetime(2014, 2, 20, 9, 0), 'HR')
# (4, 'Amitabh', 'Singh', 500000, datetime.datetime(2014, 2, 20, 9, 0), 'Admin')
# (5, 'Vivek', 'Bhati', 500000, datetime.datetime(2014, 6, 11, 9, 0), 'Admin')
# (6, 'Vipul', 'Diwan', 200000, datetime.datetime(2014, 6, 11, 9, 0), 'Account')

5
(1, 'Monika', 'Arora', 100000, datetime.datetime(2014, 2, 20, 9, 0), 'HR')
(3, 'Vishal', 'Singhal', 300000, datetime.datetime(2014, 2, 20, 9, 0), 'HR')
(4, 'Amitabh', 'Singh', 500000, datetime.datetime(2014, 2, 20, 9, 0), 'Admin')
(5, 'Vivek', 'Bhati', 500000, datetime.datetime(2014, 6, 11, 9, 0), 'Admin')
(6, 'Vipul', 'Diwan', 200000, datetime.datetime(2014, 6, 11, 9, 0), 'Account')


In [30]:
# calculate total salary of people in each department
stmt = select([worker.columns.DEPARTMENT.label('department'), 
               func.sum(worker.columns.SALARY).label('department_total')])
stmt = stmt.group_by(worker.columns.DEPARTMENT)
stmt = stmt.order_by(desc('department_total'))
results = conn.execute(stmt).fetchall()
for result in results:
    print(result.department, result.department_total)

# Admin 1170000
# HR 400000
# Account 275000

Admin 1170000
HR 400000
Account 275000


In [47]:
# another way to reflect tables
metadata = MetaData(bind=engine)
MetaData.reflect(metadata)

# reflect table
bonus = metadata.tables['Bonus']

stmt = select([bonus])
for record in conn.execute(stmt).fetchall():
    print(record)
    
# (1, 5000, datetime.datetime(2016, 2, 20, 0, 0))
# (2, 3000, datetime.datetime(2016, 6, 11, 0, 0))
# (3, 4000, datetime.datetime(2016, 2, 20, 0, 0))
# (1, 4500, datetime.datetime(2016, 2, 20, 0, 0))
# (2, 3500, datetime.datetime(2016, 6, 11, 0, 0))

(1, 5000, datetime.datetime(2016, 2, 20, 0, 0))
(2, 3000, datetime.datetime(2016, 6, 11, 0, 0))
(3, 4000, datetime.datetime(2016, 2, 20, 0, 0))
(1, 4500, datetime.datetime(2016, 2, 20, 0, 0))
(2, 3500, datetime.datetime(2016, 6, 11, 0, 0))
